In [47]:
import redis
import time
from datetime import datetime, timedelta

In [2]:
r = redis.Redis(decode_responses=True)

In [38]:
"""
set(key, value)
- set String value for String key
- passing same key would override the value by default
- TC : O(1)
"""
r.flushall()
default_set = r.set("bike:1", "Deimos")
default_set = r.set("bike:1", "Bullet")
print(default_set)  # True

default_get = r.get("bike:1")
print(default_get)  # Bullet

True
Bullet


In [34]:
"""
NX
- Only set the key if it does not exist
"""
r.flushall()
nx = r.set("bike:nx", "Deimos")
print(nx) # True
nx = r.set("bike:nx", "Bullet", nx = True)
print(nx) # None

True
None


In [37]:
"""
XX
- Only set the key if it already exists
"""
r.flushall()
xx = r.set("bike:xx1", "Deimos")
print(xx) # True
xx = r.set("bike:xx1", "BMW", xx = True)
print(xx) # True
xx = r.set("bike:xx2", "Bullet", xx = True)
print(xx) # None

True
True
None


In [41]:
"""
EX seconds
- Set the specified expire time, in seconds (a positive integer)
"""
r.flushall()
ex = r.set("bike:ex1", "Deimos", ex = 3)
time.sleep(1.5)
print(r.get("bike:ex1")) # Deimos
time.sleep(3)
print(r.get("bike:ex1")) # None

Deimos
None


In [74]:
"""
EXAT timestamp-seconds
- Set the specified Unix time at which the key will expire, in seconds (a positive integer)
"""
r.flushall()
exat = r.set("bike:exat1", "Deimos", exat = (datetime.now() + timedelta(0,3)) ) # 3 seconds from now on unix host
#  exat and pxat same for python clients since datetime is expected for both flags
time.sleep(1.5)
print(r.get("bike:exat1")) # Deimos
time.sleep(3)
print(r.get("bike:exat1")) # None

Deimos
None


In [77]:
"""
PX milliseconds
- Set the specified expire time, in milliseconds (a positive integer)
"""
r.flushall()
px = r.set("bike:px1", "Deimos", px = 3000)
time.sleep(1.5)
print(r.get("bike:px1")) # Deimos
time.sleep(3)
print(r.get("bike:px1")) # None

Deimos
None


In [79]:
"""
PXAT timestamp-milliseconds
- Set the specified Unix time at which the key will expire, in milliseconds (a positive integer)
"""
r.flushall()
pxat = r.set("bike:pxat1", "Deimos",pxat = datetime.now() + timedelta(0,3))  # 3000 milliseconds from now on unix host
#  exat and pxat same for python clients since datetime is expected for both flags
time.sleep(1.5)
print(r.get("bike:pxat1")) # Deimos
time.sleep(3)
print(r.get("bike:pxat1")) # None

Deimos
None


In [105]:
"""
KEEPTTL
- redis removes/overrides the ttl settings like EX, PX, EXAT, PXAT, etc if the latest set removes / overrides them
- Retain the time to live associated with the key
"""


# by default, keepttl is false
r.flushall()
keepttl1 = r.set("bike:keepttl1", "Deimos", ex = 3)
keepttl1 = r.set("bike:keepttl1", "Deimos") 
time.sleep(1.5)
print("Case 1: by default keep ttl settings are not retained if the latest set does not include them")
print(r.get("bike:keepttl1")) # Deimos
time.sleep(3)
print(r.get("bike:keepttl1")) # Deimos


r.flushall()
keepttl1 = r.set("bike:keepttl1", "Deimos", ex = 3)
keepttl1 = r.set("bike:keepttl1", "Deimos", ex = 1) 
print("Case 2: by default keep ttl settings are overriden if the latest set overrides them")
time.sleep(1.5)
print(r.get("bike:keepttl1")) # None
time.sleep(3)
print(r.get("bike:keepttl1")) # None


r.flushall()
keepttl1 = r.set("bike:keepttl1", "Deimos", ex = 3)
keepttl1 = r.set("bike:keepttl1", "Deimos",keepttl = True) 
print("Case 3: keepttl flag retains ttl settings from previous set command")
time.sleep(1.5)
print(r.get("bike:keepttl1")) # Deimos
time.sleep(3)
print(r.get("bike:keepttl1")) # None

Case 1: by default keep ttl settings are not retained if the latest set does not include them
Deimos
Deimos
Case 1: by default keep ttl settings are overriden if the latest set overrides them
None
None
Case 3: keepttl flag retains ttl settings from previous set command
Deimos
None


In [115]:
"""
GET
- Return the old string stored at key, or nil if key did not exist. 
- By default get flag is set to False
"""
r.flushall()
set_get1 = r.set("bike:set_get1", "BMW", get = False)
print(set_get1) # True
set_get1 = r.set("bike:set_get1", "Deimos", get = True)
print(set_get1) # BMW
set_get1 = r.set("bike:set_get1", "Bullet", get = True) 
print(set_get1) # Deimos

"""
An error is returned and SET aborted if the value stored at key is not a string
"""
"""
r.flushall()
set_get3 = r.lpush("bike:set_get3", "BMW")
print(set_get3) # 1
set_get3 = r.set("bike:set_get3", "Deimos", get = True) # error - WRONGTYPE Operation against a key holding the wrong kind of value
"""

True
BMW
Deimos


'\n# An error is returned and SET aborted if the value stored at key is not a string\nr.flushall()\nset_get3 = r.lpush("bike:set_get3", "BMW")\nprint(set_get3) # 1\nset_get3 = r.set("bike:set_get3", "Deimos", get = True) # error - WRONGTYPE Operation against a key holding the wrong kind of value\n'

In [119]:
"""
miscellaneous
"""


"""
key holding one data type can be overriden by key holding String using set
"""
r.flushall()
set_get2 = r.lpush("bike:set_get2", "BMW")
print(set_get2) # 1
set_get2 = r.set("bike:set_get2", "Deimos")
print(set_get2, r.get("bike:set_get2")) # True Deimos


"""
vica versa is not true
"""
"""
r.flushall()
set_get2 = r.set("bike:set_get2", "Deimos")
print(set_get2) # 1
set_get2 = r.lpush("bike:set_get2", "BMW") # error - WRONGTYPE Operation against a key holding the wrong kind of value
"""


1
True Deimos


'\nr.flushall()\nset_get2 = r.set("bike:set_get2", "Deimos")\nprint(set_get2) # 1\nset_get2 = r.lpush("bike:set_get2", "BMW") # error - WRONGTYPE Operation against a key holding the wrong kind of value\n'